<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2021</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Разбор задач: Typing_2</b><br/>
    <br/>
    <font>Илария Белова</font><br/>
</center>

### Typy theory

In [10]:
Задача хорошо решена, своими словами описано понимание ошибки.

😊 "Операция сложения для int и None не определена. Mypy выдаст ошибку, т.к. функция аннотирована.",

Задача c натяжкой приемлемо решена, копипаста выхлопа mypy

😑"Unsupported operand types for + ('None' and 'int')"

    
Задачи, где были такие описание причин, не засчитала:

😕"" 
☹️"gg"
😩"abc"
😭"gwht rtsdtarhsd thshaetjh rstj"

SyntaxError: invalid syntax (<ipython-input-10-de451551b15e>, line 1)

### Problem 1

In [61]:
%%typecheck
import typing as tp

def foo(a: tp.Optional[int]) -> int:
    return a + 1

a: int = {"a": 1}.get("a", 1)
foo(a)

<string>:5: error: Unsupported operand types for + ("None" and "int")
<string>:5: note: Left operand is of type "Optional[int]"
Found 1 error in 1 file (checked 1 source file)



In [ ]:
5:
* ✅ "Не, ну кто optional с числом складывает..."
* ✅ "Check for none missing"
* ✅ "переменная 'a' все еще может быть типом None, тут надо isinstance добавить, иначе упадет"
* ✅ "Как мы знаем tp.Optional[X] = Union(X, None), поэтому мы не можем просто использовать + для объектов tp.Optional"


In [ ]:
7:
* ✅ "get without specified default returns optional value"
* ✅ "По идее get вернет Optional[int], получим ошибку присваивания'        

### Problem 2

In [8]:
%%typecheck
import typing as tp

aa = [1, "2", [3]]
aa[0] += 1
aa.append({1, 3})

<string>:5: error: Unsupported operand types for + ("object" and "int")
Found 1 error in 1 file (checked 1 source file)



In [ ]:
5:
* ✅ "В массиве не только инты лежат, какое += 1?"
* ❌ "Type of aa[0] is any. any + int is not defined" (не any, а object)
* ✅ "Так как типы в list разные, то он инициализируется как Object. У Object нет операции +="
* ❌ 'Т.к тип aa это Set[tp.Object], то майпай будет ругатся когда мы складываем инт с обджектом' (List)
* ❌ "Сложение между list и int не определено" (не достаточный аргумент)

### Problem 3

In [12]:
%%typecheck
import typing as tp


def foo(a: tp.Set[int]) -> None:
    pass

a = {1.1, 2.1, 3.1}
foo(a)
b = {1, 2, 3}
foo(b)
c = {True, False}
foo(c)

<string>:9: error: Argument 1 to "foo" has incompatible type "Set[float]"; expected "Set[int]"
<string>:13: error: Argument 1 to "foo" has incompatible type "Set[bool]"; expected "Set[int]"
Found 2 errors in 1 file (checked 1 source file)



In [ ]:
9
* ✅ "сет инвариантный, значит нельзя к более широкому приводить"
* ❌ "Функция должна принимать на вход множество из целых чисел" (не аргумент)
* ❌ "Set не ковариантный" (тогда уж не контрвариантный)
* ❌ "Функция вызывается с Set[float] а не Set[int]" (не аргумент)
* ❌ "Set ковариантен по типу элементов => не можем повышать тип при передаче аргументов в функцию" (не ковариантен)

In [ ]:
13
* ✅ "сет инвариантный, значит нельзя к более узкому приводить"
* ✅ "mutable"
* ❌ "В Set[int] передается bool" (не аргумент)
* ❌ "bool вместо int" (тоже)
* ❌ "Сет boolов - это очень странно, из-за особенностей хеширования его нельзя передать сюда" (дело не в хешировании)

### Problem 4

In [14]:
%%typecheck
import typing as tp


def foo(a: tp.AbstractSet[int]) -> None:
    pass

a = {1.1, 2.1, 3.1}
foo(a)
b = {1, 2, 3}
foo(b)
c = {True, False}
foo(c)

<string>:9: error: Argument 1 to "foo" has incompatible type "Set[float]"; expected "AbstractSet[int]"
Found 1 error in 1 file (checked 1 source file)



In [ ]:
9:
* ❌ "Set ковариантен" (нет)
* ✅ "AbstractSet ковариантен, а int подтип float, поэтому AbstractSet[float] не является подтипом AbstractSet[int]"
* ❌ "Тип строго должен быть такой как объявлен, а не сложнее (выше)" (не такой же + выше более просто тип)
* ❌ AbstractSet это read-only версия Set. По такому аргументу функция контрвариантна.Ошибка только при повышении типа." (функция то да, но мы рассуждаем про контейнер, которые ковариантен по аргументу)

### Problem 5

In [16]:
%%typecheck
import typing as tp

class A:
    pass

class B(A):
    pass

def foo(a: A) -> B:
    return a.__class__()


foo(A())
foo(B())

<string>:11: error: Incompatible return value type (got "A", expected "B")
Found 1 error in 1 file (checked 1 source file)



In [17]:
11:
* ❌ "Возвращаемое значение имеет не тип 'B', а тип 'object' (со значением 'A')" (как-то лучше б звучало, что имеет тип А)
* ✅ "Повышение типа возвращаемого значения - A вместо B (B вместо A можно бы было)"
* ✅ "Если подадим А, то будет ожидаться В, здесь возвращать с повышением типа нельзя 
      тк у В могут быть методы которых не было в А"
* ✅ "метод класс от A очевидно дает A при любом входе"
* ❌ "Неверный формат" (мм?)
* ❌ "Класс возвращаемого значения не должен повышаться" (скорее тип)
* ❌ "Возвращает инстанс сабкласса А, а надо Б" (таки точнее сказать, что инстанс класса А)

SyntaxError: invalid syntax (<ipython-input-17-e2f40bd10b4c>, line 1)

### Problem 6

In [18]:
%%typecheck
class T:
    pass

class S(T):
    pass

class A:
    VAR = T()
    
class B(A):
    VAR = S()
    
class C(B):
    VAR = T()

<string>:15: error: Incompatible types in assignment (expression has type "T", base class "B" defined the type as "S")
Found 1 error in 1 file (checked 1 source file)



In [19]:
15:
* ❌ "Происходит понижение типа VAR от T() к S() при наследовании класса C от B" (повышение от S к Т)
* ❌ "Нельзя делать override переменной класса, понижая тип" (повышая)
* ✅ "повышение типа для override-переменной"
* ❌ "В качестве типа для VAR может использоваться лишь тип S" (ну и подтипы)
* ✅ "нельзя повысить тип, так как у B могут быть использованы методы, "
            "которые используют методы S, а класс С имеет все методы В, поэтому от типа Т могут вызваться методы S
* ❌ "Нельзя ж" (ну да)
* ❌ 'Сужение типа' (расширение)

SyntaxError: invalid syntax (<ipython-input-19-84908d002a91>, line 1)

### Problem 7

In [20]:
%%typecheck
import typing as tp

class A:
    pass

class B(A):
    pass

def g(f: tp.Callable[[A], B]) -> None:
    pass

def f1(a: A) -> A:
    pass

def f2(a: A) -> B:
    pass

def f3(a: B) -> A:
    pass

def f4(a: B) -> B:
    pass

g(f1)
g(f2)
g(f3)
g(f4)

<string>:25: error: Argument 1 to "g" has incompatible type "Callable[[A], A]"; expected "Callable[[A], B]"
<string>:27: error: Argument 1 to "g" has incompatible type "Callable[[B], A]"; expected "Callable[[A], B]"
<string>:28: error: Argument 1 to "g" has incompatible type "Callable[[B], B]"; expected "Callable[[A], B]"
Found 3 errors in 1 file (checked 1 source file)



In [ ]:
✅
25: 'повышение типа возвращаемого значения',
27: 'повышение типа возвращаемого значения, понижение типа аргумента',
28: 'понижение типа аргумента',
    
✅
25: "Объекты Callable ковариантны по значению. Нельзя повышать тип значения!",
27: "Нельзя вообще ничего, Наташ, честно!",
28: "Объекты Callable контрвариантны по аргументам. Нельзя понижать тип аргумента!",

❌ (необъяснено, в чем неверность то)
25: "Функция возвращает аргумент неверного типа", (значение уж)
27: "Функция принимает аргумент неверного типа", (и значение)
28: "Функция принимает аргумент неверного типа"

❌ (не аргумент)
25: "Ждем тип Callable[[A], B]",
27: "Ждем тип Callable[[A], B]",
28: "Ждем тип Callable[[A], B]"


### Problem 8

In [22]:
%%typecheck
import typing as tp


def foo(a: tp.Iterable[str]) -> bool:
    b = len(a)
    c = sum(1 for i in a)
    return b == c

foo(["a", "b"])
foo("ab")
foo({"a": 2})

class A:
    def __len__(self) -> int:
        return 1

foo(A())

class B:
    def __iter__(self) -> tp.Iterator[int]:
        return iter([])
    
foo(B())


class C:
    def __iter__(self) -> tp.Iterator[str]:
        return iter([])
    
foo(C())

<string>:6: error: Argument 1 to "len" has incompatible type "Iterable[str]"; expected "Sized"
<string>:18: error: Argument 1 to "foo" has incompatible type "A"; expected "Iterable[str]"
<string>:24: error: Argument 1 to "foo" has incompatible type "B"; expected "Iterable[str]"
<string>:24: note: Following member(s) of "B" have conflicts:
<string>:24: note:     Expected:
<string>:24: note:         def __iter__(self) -> Iterator[str]
<string>:24: note:     Got:
<string>:24: note:         def __iter__(self) -> Iterator[int]
Found 3 errors in 1 file (checked 1 source file)



In [ ]:
✅
6: "Iterable не поддерживает метод __len__()",
18: "A не является Iterable",
24: "B Iterable[int] хотелось бы Iterable[str]"
    
✅
6: "len() не определен для Iterable, нужен Sized",
18: "Функция ожидаем Iterable[str], а не Sized",
24: "Функция ожидаем Iterable[str], а не Iterable[int]"
    
✅
6: 'Протокол Iterable не гарантирует реализацию dunder метода __len__',
18: 'Не реализован протокол Iterable[str], который подразумевает реализацию в классе dunder метода __iter__() '
            '-> Iterable[str]',
24: 'Реализоан протокол Iterable[int], а не Iterable[str], а типы str не является надтипом int также как int '
            'не является надтипом str'

### Problem 9

In [23]:
%%typecheck
import typing as tp

class Fooable(tp.Protocol, tp.Sized):
    def foo(self, a: int) -> None:
        pass


class A(Fooable):
    def __len__(self) -> int:
        return 10

class B:
    def foo(self, a: int) -> None:
        pass

    def __len__(self) -> int:
        return 10

class C:
    def foo(self, a: int) -> None:
        pass


def foo(a: int) -> None:
    pass


def f(c: Fooable) -> None:
    c.foo(1)
    len(c)
    "10" in c

f([])
f(A())
f(B())
f(C())
f(foo)

<string>:32: error: Unsupported right operand type for in ("Fooable")
<string>:34: error: Argument 1 to "f" has incompatible type "List[<nothing>]"; expected "Fooable"
<string>:34: note: 'list' is missing following 'Fooable' protocol member:
<string>:34: note:     foo
<string>:37: error: Argument 1 to "f" has incompatible type "C"; expected "Fooable"
<string>:37: note: 'C' is missing following 'Fooable' protocol member:
<string>:37: note:     __len__
<string>:38: error: Argument 1 to "f" has incompatible type "Callable[[int], None]"; expected "Fooable"
Found 4 errors in 1 file (checked 1 source file)



In [ ]:
✅
32: "У Fooable нет метода __contains__()",
34: "[] не является Fooable (нет метода foo())",
37: "C не является Fooable (нет метода __len__())",
38: "foo не является Fooable (нет метода __len__())"
    
❌
32: "У Fooable нет метода in", (in не метод)
34: "Ожидалось Fooable", (конкретика не помешала бы)
37: "Ожидалось Fooable",
38: "Ожидалось Fooable"}

### Problem 10

In [24]:
%%typecheck
import typing as tp

T = tp.TypeVar("T", bound=tp.SupportsFloat, covariant=True)

class A(tp.Generic[T]):
    def __init__(self, a: T) -> None:
        self._a: tp.SupportsFloat = a
        
    def increment(self) -> float:
        self._a = float(self._a) + 1
        return self._a
    

A(1)
A(1.2)
A(True)
A("1.3")

class B:
    def __float__(self) -> float:
        return 1.1
    
A(B())

def g(a: A[int]) -> None:
    pass

g(A(1.4))
g(A(True))

<string>:18: error: Value of type variable "T" of "A" cannot be "str"
<string>:29: error: Argument 1 to "A" has incompatible type "float"; expected "int"
Found 2 errors in 1 file (checked 1 source file)



In [ ]:
✅
18: "Как выяснилось, у str нет аттрибута __float__",
29: "В строчке 30 всё хорошо, т.е. bool - подкласс int и шаблонный тип T ковариантен, тут же float - родитель int."

❌
18: "Неправильный тип, не может быть тип str.",
29: "Нужен тип int." (ну не только int, но и подтипы)

In [ ]:
"(КАК Я УСТАЛЛ!!! ЗАЧЕМ  ЯСЮДА ПРИШЁЛ!!!)"
    
Держись!

### Typy annotations

In [ ]:
# Лишний bound

import typing as tp


def f(a: float) -> float:
    return a / 2

In [ ]:
# Лишняя ковариантность

import typing as tp

T1 = tp.TypeVar("T1", bound=str)
T2 = tp.TypeVar("T2", str, tp.List[str])


def f(a: tp.Tuple[tp.Sized, T1, T2]) -> T1:
    return a[1]

In [ ]:
# Лишняя контрвариантность

import typing as tp

T1 = tp.TypeVar("T1")
T2 = tp.TypeVar("T2")
T3 = tp.TypeVar("T3")
S = tp.TypeVar("S", int, bool, float)


def f(a: tp.Callable[[T1, T2, T3], S], b: T1, c: T2, d: T3) -> S:
    return a(b, c, d)


### Typy protocol

In [ ]:
import typing as tp


T_co = tp.TypeVar('T_co', covariant=True)


class Gettable(tp.Protocol[T_co]):
    def __getitem__(self, item: int) -> T_co:
        pass

    def __len__(self) -> int:
        pass


def get(container: Gettable[T_co], index: int) -> tp.Optional[T_co]:
    if container:
        return container[index]

    return None

<div class="alert alert-danger">
<b>Антипаттерн:</b> именовать ковариантные параметры как T (а не T_co)
</div>

In [ ]:

T_co = tp.TypeVar('T', covariant=True)


class Gettable(Protocol[T_co]):
    def __getitem__(self, item: int) -> T:
        pass

    def __len__(self) -> int:
        pass


def get(container: Gettable[T], index: int) -> tp.Optional[T]:
    if container:
        return container[index]

    return None

<div class="alert alert-danger">
<b>Антипаттерн:</b> реализовывать протоколы
</div>

In [ ]:
import typing as tp
from typing_extensions import Protocol  # В 3.8 уже из коробки

T_co = tp.TypeVar('T_co', covariant=True)


class Gettable(Protocol[T_co]):
    def __len__(self) -> int: ...

    def __getitem__(self, key: tp.Any) -> tp.Any: ...


def get(container: Gettable[str], index: int) -> tp.Optional[str]:
    if container:
        return container[index]
    return None

<div class="alert alert-danger">
<b>Антипаттерн:</b> не пользоваться тем, что Protocol уже дженерик в иерархии
</div>

In [ ]:
import typing as tp

T = tp.TypeVar('T', covariant=True)


class Gettable(tp.Protocol[T]):
    def __getitem__(self, item: int) -> T: pass

    def __len__(self) -> int: pass


def get(container: Gettable[T], index: int) -> tp.Optional[T]:
    if container:
        return container[index]
    return None

### Typy generic

In [46]:
import typing as tp

T = tp.TypeVar('T', int, float)


class Pair(tp.Generic[T]):
    def __init__(self, a: T, b: T) -> None:
        self._a: T = a
        self._b: T = b

    def sum(self) -> T:
        return self._a + self._b

    def first(self) -> T:
        return self._a

    def second(self) -> T:
        return self._b

    def __iadd__(self, pair: 'Pair[T]') -> 'Pair[T]':
        self._a += pair.first()
        self._b += pair.second()
        return self


<div class="alert alert-danger">
<b>Антипаттерн:</b> мутабельные ковариантные типы
</div>

In [65]:
%%typecheck

import typing as tp


T = tp.TypeVar("T", int, float)


class Pair(tp.Generic[T]):
    def __init__(self, a: T, b: T) -> None:
        self.x: T = a
        self.y: T = b

    def sum(self) -> T:
        return self.x + self.y

    def first(self) -> T:
        return self.x

    def second(self) -> T:
        return self.y

    def __iadd__(self, pair: 'Pair[T]') -> 'Pair[T]':
        self.x += pair.first()
        self.y += pair.second()
        return self
    

def f(p: Pair[float]) -> None:
    p += Pair(1.1, 2.1)
    
p: Pair[int] = Pair(1, 2)
f(p)

p.first() >> 1

<string>:33: error: Argument 1 to "f" has incompatible type "Pair[int]"; expected "Pair[float]"
Found 1 error in 1 file (checked 1 source file)

